In [ ]:
import pandas as pd
import torch
import ast
# from sentence_transformers import SentenceTransformer
from sentence_transformers import util

In [1]:
# @title Load Configuraiton 
import hbqaconfig
kaggle=False
conf = hbqaconfig.setEnv('local')

In [6]:
df_chunk = pd.read_csv(conf['QAGS_FOLDER'] + '06.10-ChatGPTAPI_Chunk_Vector_multi-qa-mpnet-base-dot-v1.csv')
df_qa = pd.read_csv(conf['QAGS_FOLDER'] + '06.12-ChatGPTAPI_QA_Vector_multi-qa-mpnet-base-dot-v1.csv')

In [7]:
df_chunk.shape, df_qa.shape

((285, 4), (1084, 10))

In [15]:
df_chunk.head(3)

,Chunk_Id,Section_Id,Chunk,ChunkVector_multi-qa-mpnet-base-dot-v1
0,0,Book01_002,THE MAHABHARATA ADI PARVA Section I\nOm! Havin...,"[-0.014535784721374512, -0.08219598978757858, ..."
1,1,Book01_002,The Rishi Vyasa published this mass of knowled...,"[0.2731589078903198, -0.15328504145145416, -0...."
2,2,Book01_002,"Vyasa executed the compilation of the Bharata,...","[-0.019269350916147232, -0.4155203402042389, -..."


In [9]:
df_qa.head(3)

,Ques_Id,Chunk_Id,Section_Id,Question,Ref_Answer,Chunk,WordsInQues,WordsInAns,QuestionVector_multi-qa-mpnet-base-dot-v1,Ref_AnswerVector_multi-qa-mpnet-base-dot-v1
0,0,0,Book01_002,Who asked Sauti about his journey?,One of the Rishis beginning the conversation a...,THE MAHABHARATA ADI PARVA Section I\nOm! Havin...,6,12,"[0.2826710641384125, -0.3960507810115814, -0.3...","[0.028256800025701523, -0.1837993562221527, -0..."
1,1,0,Book01_002,What did the Rishis wish to hear from Sauti?,The Rishis wished to hear the wonderful narrat...,THE MAHABHARATA ADI PARVA Section I\nOm! Havin...,9,10,"[0.47221648693084717, -0.4937230348587036, -0....","[0.246647909283638, -0.41256460547447205, -0.3..."
2,2,0,Book01_002,What did the Rishis ask Sauti to recite?,The Rishis asked Sauti to recite the sacred st...,THE MAHABHARATA ADI PARVA Section I\nOm! Havin...,8,32,"[0.4244302809238434, -0.7060792446136475, -0.3...","[0.21940578520298004, -0.16441603004932404, -0..."


In [3]:
#Select Model Function

# https://www.sbert.net/docs/pretrained_models.html

#250MB, multi-qa-distilbert-cos-v1',  Max Sequence Length:	512, Dimensions:768, Normalized Embeddings:	true
#80MB, all-MiniLM-L6-v2, Max Sequence Length:	256, Dimensions:	384, Normalized Embeddings:	true
#290MB, all-distilroberta-v1, Max Sequence Length:	512, Dimensions:	768, Normalized Embeddings:	true
#420MB, all-mpnet-base-v2, Max Sequence Length:	384, Dimensions:	768, Normalized Embeddings:	true
#1.36GB, all-roberta-large-v1, Max Sequence Length:	256, Dimensions: 1024, Normalized Embeddings:	true

def select_embmodel(num):
    emb_modelshortlist = ['distilbert','minilm','distilroberta','mpnet','roberta']

    emb_modellist = ['multi-qa-distilbert-cos-v1',
                'all-MiniLM-L6-v2',
                'all-distilroberta-v1',
                'multi-qa-mpnet-base-dot-v1',
                'all-roberta-large-v1']

    embmodelname = emb_modellist[num]
    embmodelshort = emb_modelshortlist[num]
    embmodelname1 = "_" + embmodelname

    print (embmodelname,'\t',embmodelshort,'\t', embmodelname1)
    return embmodelname, embmodelshort, embmodelname1

embmodelname, embmodelshort, embmodelname1 = select_embmodel(3)

multi-qa-mpnet-base-dot-v1 	 mpnet 	 _multi-qa-mpnet-base-dot-v1


In [8]:
embmodel = SentenceTransformer(embmodelname)

In [9]:
df_chunk.head(2)

,Chunk_Id,Section_Id,Chunk,ChunkVector_multi-qa-mpnet-base-dot-v1
0,0,Book01_002,THE MAHABHARATA ADI PARVA Section I\nOm! Havin...,"[-0.014535784721374512, -0.08219598978757858, ..."
1,1,Book01_002,The Rishi Vyasa published this mass of knowled...,"[0.2731589078903198, -0.15328504145145416, -0...."


In [14]:
chunk_id = df_chunk['Chunk_Id'].to_list()

chunk_vector = [ast.literal_eval(df_chunk['ChunkVector'+embmodelname1][i]) for i in range(df_chunk.shape[0])]
chunk_vector = torch.tensor(chunk_vector)

In [12]:
df_qa.index=df_qa.Ques_Id
df_qa.head(2)

,Ques_Id,Chunk_Id,Section_Id,Question,Ref_Answer,Chunk,WordsInQues,WordsInAns,QuestionVector_multi-qa-mpnet-base-dot-v1,Ref_AnswerVector_multi-qa-mpnet-base-dot-v1
Ques_Id,,,,,,,,,,
0,0,0,Book01_002,Who asked Sauti about his journey?,One of the Rishis beginning the conversation a...,THE MAHABHARATA ADI PARVA Section I\nOm! Havin...,6,12,"[0.2826710641384125, -0.3960507810115814, -0.3...","[0.028256800025701523, -0.1837993562221527, -0..."
1,1,0,Book01_002,What did the Rishis wish to hear from Sauti?,The Rishis wished to hear the wonderful narrat...,THE MAHABHARATA ADI PARVA Section I\nOm! Havin...,9,10,"[0.47221648693084717, -0.4937230348587036, -0....","[0.246647909283638, -0.41256460547447205, -0.3..."


In [15]:
# Search Chunk based on Each Question. Get 5 chunk for each question
# for i in df_qa.index:
df_pred = pd.DataFrame(columns = ['Ques_Id','Chunk_Id','Pred_Chunk_Id'])

for i in df_qa.index:
  # query = df_qa.loc[i,'Question']
  # question_embedding = torch.tensor(embmodel.encode(query).tolist())
  question_embedding = ast.literal_eval( df_qa.loc[ i,'QuestionVector' + embmodelname1])  
  question_embedding = torch.tensor(question_embedding)

  # print (type(question_embedding), question_embedding)
  top_k=10
  hits = util.semantic_search(question_embedding, chunk_vector, top_k=top_k)


  predictedId= [item['corpus_id'] for item in hits[0]]
  # print (hits[0])
  df_pred.loc[i]= (df_qa.loc[i,'Ques_Id'], df_qa.loc[i,'Chunk_Id'], predictedId)
  print('Predicting: ',i)
  # break

Predicting:  0
Predicting:  1
Predicting:  2
Predicting:  3
Predicting:  4
Predicting:  5
Predicting:  6
Predicting:  7
Predicting:  8
Predicting:  9
Predicting:  10
Predicting:  11
Predicting:  12
Predicting:  13
Predicting:  14
Predicting:  15
Predicting:  16
Predicting:  17
Predicting:  18
Predicting:  19
Predicting:  20
Predicting:  21
Predicting:  22
Predicting:  23
Predicting:  24
Predicting:  25
Predicting:  26
Predicting:  27
Predicting:  28
Predicting:  29
Predicting:  30
Predicting:  31
Predicting:  32
Predicting:  33
Predicting:  34
Predicting:  35
Predicting:  36
Predicting:  37
Predicting:  38
Predicting:  39
Predicting:  40
Predicting:  41
Predicting:  42
Predicting:  43
Predicting:  44
Predicting:  45
Predicting:  46
Predicting:  47
Predicting:  48
Predicting:  49
Predicting:  50
Predicting:  51
Predicting:  52
Predicting:  53
Predicting:  54
Predicting:  55
Predicting:  56
Predicting:  57
Predicting:  58
Predicting:  59
Predicting:  60
Predicting:  61
Predicting:  62
Pr

In [17]:
df_pred.to_csv(conf['RAAGS_FOLDER'] + '10.1-Predict-DocumentId-for-Ques'+embmodelname1 )

In [35]:
# Mean Reciprocal Rank (MRR) is a metric commonly used to evaluate the performance of ranking-based retrieval systems,
# such as search engines. MRR assesses how well a retrieval system ranks relevant items by considering the position of the
# first relevant item in the ranked list. It's calculated as the average of the reciprocal ranks across a set of queries.

def getReciprocalRank(row):
  K=1 # Number of relevent document
  try:   
    i = row['Pred_Chunk_Id'].index( row['Chunk_Id']) #ast.literal_eval( row['Pred_Chunk_Id']).index(row['Chunk_Id'] )
  except:
    i=-1

  if i==-1:
    RR=0
  else:
    RR = round(1/(i + 1),2)

#   print (i)
  return RR

df_pred['RR'] = df_pred.apply(getReciprocalRank, axis=1)

In [37]:
df_pred.sample(10)

,Ques_Id,Chunk_Id,Pred_Chunk_Id,RR
668,668,157,"[205, 37, 197, 33, 228, 43, 66, 147, 127, 248]",0.0
940,940,240,"[277, 225, 128, 131, 74, 91, 144, 267, 232, 139]",0.0
843,843,210,"[24, 25, 176, 229, 31, 28, 30, 154, 126, 119]",0.0
581,581,132,"[116, 160, 235, 169, 161, 162, 236, 27, 209, 142]",0.0
637,637,146,"[183, 11, 7, 77, 2, 54, 10, 178, 243, 217]",0.0
207,207,42,"[34, 137, 174, 149, 146, 39, 283, 206, 143, 102]",0.0
729,729,179,"[42, 7, 44, 184, 147, 0, 139, 143, 280, 138]",0.0
91,91,19,"[284, 198, 196, 194, 199, 190, 255, 204, 193, 16]",0.0
764,764,188,"[116, 236, 235, 142, 168, 169, 218, 160, 162, ...",0.0
380,380,77,"[235, 112, 170, 23, 29, 144, 226, 139, 236, 113]",0.0


In [39]:
df_pred['RR'].mean()

0.0189760147601476

In [ ]:
# Summarise Above 5 Chunk using lang chain

In [ ]:
# Load t5_30epoch finetuned model. 
# Use Summarise Chunk and Question to Find Answer Using AGS
